In [2]:
import numpy as np
import pandas as pd
import gzip
import re
from urllib.parse import urlparse, parse_qs
import os

In [3]:
nbgitpuller_df = pd.read_json(gzip.open('../spring-2023/nbgitpuller-clicks-spring-2023.json.gz'), lines = True)
nbgitpuller_df.head()

,insertId,labels,logName,receiveTimestamp,resource,severity,textPayload,timestamp
0,zfcnwk0zwtv1iq25,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-01-01T00:01:21.752824112Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-01-01 00:01:18.743 JupyterHub log:186]...,2023-01-01 00:01:18.744083889+00:00
1,z7c104em5irpurin,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-01-01T00:07:09.570118943Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-01-01 00:07:08.123 JupyterHub log:186]...,2023-01-01 00:07:08.123614954+00:00
2,3a8drkn15kv63vcy,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-01-01T00:07:09.570118943Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-01-01 00:07:09.177 JupyterHub log:186]...,2023-01-01 00:07:09.177935010+00:00
3,473ix0agod0y0uyx,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-01-01T00:07:49.571954687Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-01-01 00:07:44.794 JupyterHub log:186]...,2023-01-01 00:07:44.794346370+00:00
4,v8xyt3k2ovdxxaz4,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-01-01T00:09:46.721741684Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-01-01 00:09:43.333 JupyterHub log:186]...,2023-01-01 00:09:43.333464197+00:00


In [41]:
nbgitpuller_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623529 entries, 0 to 623528
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype              
---  ------            --------------   -----              
 0   insertId          623529 non-null  object             
 1   labels            623529 non-null  object             
 2   logName           623529 non-null  object             
 3   receiveTimestamp  623529 non-null  object             
 4   resource          623529 non-null  object             
 5   severity          623529 non-null  object             
 6   textPayload       623529 non-null  object             
 7   timestamp         623529 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), object(7)
memory usage: 38.1+ MB


In [4]:
nbgitpuller_df.insertId.value_counts()
# how to figure out which ones are the same user?

c0v13bw0dxpdbys8    3
kjiom6dco4qdl5tl    2
9fnr0t7yub8fgtr8    2
gkw17rk5wr7qy751    2
x10rzxtolgz0ykuc    2
                   ..
70zfk9m4cqzqqv1w    1
0u42pkppmo4spnko    1
2w2yxbw8rpelx72b    1
bu7dpps6l4awrcf0    1
olzadu77iox2npd6    1
Name: insertId, Length: 623511, dtype: int64

In [5]:
nbgitpuller_df.textPayload[0]

'[I 2023-01-01 00:01:18.743 JupyterHub log:186] 302 GET /hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fdata-8%2Fmaterials-fa22&urlpath=retro%2Ftree%2Fmaterials-fa22%2Fmaterials%2Ffa22%2Flab%2Flab01%2Flab01.ipynb&branch=main -> /hub/login?next=%2Fhub%2Fuser-redirect%2Fgit-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fdata-8%252Fmaterials-fa22%26urlpath%3Dretro%252Ftree%252Fmaterials-fa22%252Fmaterials%252Ffa22%252Flab%252Flab01%252Flab01.ipynb%26branch%3Dmain (@10.0.0.6) 0.98ms'

In [6]:
# need to figure out the log level - do we care about this?
sample = nbgitpuller_df.textPayload[0]
sample

'[I 2023-01-01 00:01:18.743 JupyterHub log:186] 302 GET /hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fdata-8%2Fmaterials-fa22&urlpath=retro%2Ftree%2Fmaterials-fa22%2Fmaterials%2Ffa22%2Flab%2Flab01%2Flab01.ipynb&branch=main -> /hub/login?next=%2Fhub%2Fuser-redirect%2Fgit-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Fdata-8%252Fmaterials-fa22%26urlpath%3Dretro%252Ftree%252Fmaterials-fa22%252Fmaterials%252Ffa22%252Flab%252Flab01%252Flab01.ipynb%26branch%3Dmain (@10.0.0.6) 0.98ms'

In [7]:
log_info = nbgitpuller_df.textPayload.apply(lambda x: ''.join(re.findall("\[.*\]", x)).replace('[', '').replace(']', '').split(' '))


In [8]:
log_info[0]

['I', '2023-01-01', '00:01:18.743', 'JupyterHub', 'log:186']

In [9]:
# obtaining substring after GET and before the redirection
urls = nbgitpuller_df.textPayload.apply(lambda x: x[x.find('GET')+3:x.find('->')].strip())
urls[0]

'/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fdata-8%2Fmaterials-fa22&urlpath=retro%2Ftree%2Fmaterials-fa22%2Fmaterials%2Ffa22%2Flab%2Flab01%2Flab01.ipynb&branch=main'

In [10]:
urls_parsed = urls.apply(lambda x: urlparse(x))

In [11]:
actions = urls_parsed.apply(lambda x: os.path.basename(x.path))

In [14]:
url_queries = urls_parsed.apply(lambda x: parse_qs(x.query))

{'repo': ['https://github.com/data-8/materials-fa22'],
 'urlpath': ['retro/tree/materials-fa22/materials/fa22/lab/lab01/lab01.ipynb'],
 'branch': ['main']}

In [29]:
url_queries[0]['repo']

['https://github.com/data-8/materials-fa22']

In [36]:
repos = url_queries.apply(lambda x: x.get('repo')[0] if x.get('repo') else None)

In [43]:
repos.info()
# there appears to be 9 log entries without a repo name

<class 'pandas.core.series.Series'>
RangeIndex: 623529 entries, 0 to 623528
Series name: textPayload
Non-Null Count   Dtype 
--------------   ----- 
623520 non-null  object
dtypes: object(1)
memory usage: 4.8+ MB


In [55]:
url_queries[repos.isna()]

112244    {'account': ['class-account'], 'path': ['RC_La...
203505    {'reset?repo': ['https://github.com/DS-100/sp2...
215170    {' repo': ['https://github.com/ds-modules/SW-2...
277827                          {'account': ['ds-modules']}
310073                                                   {}
310075                                                   {}
446591    {'account': ['LogsCrepo=EEP153-Project3-Spring...
446593    {'account': ['LogsCrepo=EEP153-Project3-Spring...
515128                                                   {}
Name: textPayload, dtype: object

In [58]:
nbgitpuller_df[repos.isna()]

,insertId,labels,logName,receiveTimestamp,resource,severity,textPayload,timestamp
112244,x0wty2193og1j9ym,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-02-01T22:38:59.829577054Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-02-01 22:38:59.759 JupyterHub log:186]...,2023-02-01 22:38:59.759808676+00:00
203505,akhldxkk8bqprqe9,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-02-15T18:07:21.891237575Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-02-15 18:07:20.570 JupyterHub log:186]...,2023-02-15 18:07:20.570482563+00:00
215170,41vvuoddeerzds7j,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-02-16T22:26:31.933617587Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-02-16 22:26:30.758 JupyterHub log:186]...,2023-02-16 22:26:30.758262870+00:00
277827,b0r85kgp3pi8n7wk,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-02-27T21:17:01.934213097Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-02-27 21:17:00.566 JupyterHub log:186]...,2023-02-27 21:17:00.567655853+00:00
310073,bxvludlr9sboev3k,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-03-05T20:48:46.96765191Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-03-05 20:48:44.829 JupyterHub log:186]...,2023-03-05 20:48:44.830023537+00:00
310075,arqovpme4dahkr1i,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-03-05T20:48:51.879211365Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-03-05 20:48:46.912 JupyterHub log:186]...,2023-03-05 20:48:46.912822045+00:00
446591,i2846ttzskvx9w6m,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-04-05T20:44:25.992645698Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-04-05 20:44:22.856 JupyterHub log:191]...,2023-04-05 20:44:22.857108156+00:00
446593,cwubfr714ozhctkx,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-04-05T20:44:31.037794207Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-04-05 20:44:27.192 JupyterHub log:191]...,2023-04-05 20:44:27.192898761+00:00
515128,b02tzu2dptq5fuqk,{'compute.googleapis.com/resource_name': 'gke-...,projects/ucb-datahub-2018/logs/stderr,2023-04-18T06:05:35.925587637Z,"{'labels': {'cluster_name': 'fall-2019', 'cont...",ERROR,[I 2023-04-18 06:05:35.334 JupyterHub log:191]...,2023-04-18 06:05:35.334815871+00:00


In [63]:
missing_repo_entries = nbgitpuller_df.textPayload[repos.isna()]

In [64]:
missing_repo_entries

112244    [I 2023-02-01 22:38:59.759 JupyterHub log:186]...
203505    [I 2023-02-15 18:07:20.570 JupyterHub log:186]...
215170    [I 2023-02-16 22:26:30.758 JupyterHub log:186]...
277827    [I 2023-02-27 21:17:00.566 JupyterHub log:186]...
310073    [I 2023-03-05 20:48:44.829 JupyterHub log:186]...
310075    [I 2023-03-05 20:48:46.912 JupyterHub log:186]...
446591    [I 2023-04-05 20:44:22.856 JupyterHub log:191]...
446593    [I 2023-04-05 20:44:27.192 JupyterHub log:191]...
515128    [I 2023-04-18 06:05:35.334 JupyterHub log:191]...
Name: textPayload, dtype: object

In [67]:
missing_repo_entries[203505]

'[I 2023-02-15 18:07:20.570 JupyterHub log:186] 302 GET /hub/user-redirect/git-pull?reset?repo=https%3A%2F%2Fgithub.com%2FDS-100%2Fsp23&branch=main&urlpath=lab%2Ftree%2Fsp23%2Fhw%2Fhw02%2Fhw02.ipynb -> /hub/spawn/rohan.jha?next=%2Fuser%2Frohan.jha%2Fgit-pull%3Freset%253Frepo%3Dhttps%253A%252F%252Fgithub.com%252FDS-100%252Fsp23%26branch%3Dmain%26urlpath%3Dlab%252Ftree%252Fsp23%252Fhw%252Fhw02%252Fhw02.ipynb (rohan.jha@10.0.0.23) 19.61ms'

In [ ]:
# create df of the log_info_types timestamp, the action (pull or sync), and then the repo and make it into a dataframe
nbgitpuller_textPayload_df = pd.DataFrame({'log_info_type': log_info.apply(lambda x: x[0]),
                                           'timestamp_date': log_info.apply(lambda x: x[1]),
                                           'timestamp_time': log_info.apply(lambda x: x[2]),
                                           'action': actions,
                                           'repo': repos,
                                           'hub': ,
                                           'semester': })

In [24]:
nbgitpuller_df_sample = nbgitpuller_df.sample(100)

In [23]:
def strip_find_all(result, url):
    if len(result) != 0:
        result = result[0]
        return result
    else:
        print("URL", url)
        print("RESULT", result)
        return []


def find_repo_name2(url):
    action = ""
    if 'git-sync' in url or 'git-pull' in url:
        if 'git-sync' in url:
            action = 'sync'
        elif 'git-pull' in url:
            action = 'pull'

        if 'github.com' in url:
            result = re.findall("github\.com(\/|%2F|%252F)(.+?)(\/|%2F|%252F)(.+?)&", url)
            if len(result) == 0:
                result = re.findall("github\.com(\/|%2F|%252F)(.+?)(\/|%2F|%252F)(.+)", url)
            result = strip_find_all(result, url)
            if len(result) == 0:
                return "", "", "failed"
            return result[1], result[3], action
        elif "github.berkeley.edu" in url:
            result = re.findall("github\.berkeley\.edu(\/|%2F|%252F)(.+?)(\/|%2F|%252F)(.+?)&", url)
            if len(result) == 0:
                result = re.findall("github\.berkeley\.edu(\/|%2F|%252F)(.+?)(\/|%2F|%252F)(.+)", url)
            result = strip_find_all(result, url)
            if len(result) == 0:
                return "", "", "failed"
            return result[1], result[3], action
        elif "github" in url:
            result = re.findall("github(\.|%2F)com(\/|%2F|%252F)(.+?)(\/|%2F|%252F)(.+?)&", url)
            if len(result) == 0:
                result = re.findall("github(\.|%2F)com(\/|%2F|%252F)(.+?)(\/|%2F|%252F)(.+)", url)
            result = strip_find_all(result, url)
            if len(result) == 0:
                return "", "", "failed"
            return result[2], result[4], action
        elif 'git@' in url:
            result = re.findall("git@(.+?):(.+?)(\/|%2F|%252F)(.+)", url)
            result = strip_find_all(result, url)
            if len(result) == 0:
                return "", "", "failed"
            return result[1], result[3], action
    else:
        if 'account=' in url and 'repo=' in url:
            result = re.findall("account=(.+?)&.*repo=(.+?)&", url)
            if len(result) == 0:
                result = re.findall("account=(.+?)&.*repo=(.+)", url)
        elif 'account=' in url:
            result = re.findall("account=(.+)", url)
        elif 'repo=' in url:
            result = re.findall("repo=(.+?)&", url)
            if len(result) == 0:
                result = re.findall("repo=(.+)", url)
        else:
            print("URL", url)
            return "", "", "failed"
        action = "interact"
        result = strip_find_all(result, url)
        if len(result) == 0:
            return "", "", "failed"
        if 'account=' in url and 'repo=' in url:
            return result[0], result[1], action
        elif 'account=' in url:
            return result[0], "", action
        elif 'repo=' in url:
            return "", result[0], action
    return "", "", "failed"
        

In [27]:
extracted = nbgitpuller_df_sample['textPayload'].apply(find_repo_name2)

In [28]:
extracted

76742      (data-8, materials-sp23, pull)
36371                (DS-100, sp23, pull)
49313      (data-8, materials-sp20, sync)
180263    (prob140, materials-sp23, pull)
228001               (DS-100, sp23, pull)
                       ...               
70790      (data-8, materials-sp23, pull)
482640     (data-8, materials-sp23, pull)
452588     (data-8, materials-sp23, pull)
32014       (ph142-ucb, ph142-sp23, pull)
173390      (ph142-ucb, ph142-sp23, pull)
Name: textPayload, Length: 100, dtype: object